<a href="https://colab.research.google.com/github/ikramMc/PFE/blob/main/model_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Installing Required Packages**

In [ ]:
%%capture
!pip install wandb
!pip install unsloth

# **Model loading and lora configuration**

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length =4700
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.
model_name = "unsloth/Llama-3.1-8B-unsloth-bnb-4bit"
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name =model_name,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 32, #Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules =["q_proj", "k_proj", "v_proj", "o_proj","gate_proj", "up_proj", "down_proj",],
    lora_alpha = 64,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized,don't train biases only weights
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

# **Data Preparation**

We use get_chat_template function to get the correct chat template. unsloth support zephyr, chatml, mistral, llama, llama3.1 alpaca, vicuna, vicuna_old ...



In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

def formatting_prompts_func(examples):
   convos = examples["conversations"]
   texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False).removeprefix('<|begin_of_text|>') for convo in convos]
   return { "text" : texts, }
pass


In [ ]:
from datasets import Dataset
from unsloth.chat_templates import standardize_sharegpt
import pandas as pd
import ast

df = pd.read_csv("train.csv")

def parse_conversations(conv_str):
    try:
        return ast.literal_eval(conv_str)
    except:
        return []

# Transformer en liste de dictionnaires
dataset_list = [
    {"conversations": parse_conversations(conv)} for conv in df["conversations"]
]
hf_dataset = Dataset.from_list(dataset_list)



In [ ]:
# Remove empty conversations
hf_dataset = hf_dataset.filter(
    lambda example: example["conversations"] and len(example["conversations"]) > 0
)


In [ ]:
print(hf_dataset[12]["conversations"])

We now use `standardize_sharegpt` to convert ShareGPT style datasets into HuggingFace's generic format. This changes the dataset from looking like:
```
{"from": "system", "value": "You are an assistant"}
{"from": "human", "value": "What is 2+2?"}
{"from": "gpt", "value": "It's 4."}
```
to
```
{"role": "system", "content": "You are an assistant"}
{"role": "user", "content": "What is 2+2?"}
{"role": "assistant", "content": "It's 4."}
```
in our case our data is already saved in huggingFace's generic format

In [ ]:
#to hugging face formart(rule,content)
from unsloth.chat_templates import standardize_sharegpt
train_dataset = standardize_sharegpt(hf_dataset)
train_dataset = train_dataset.map(formatting_prompts_func, batched = True,)#apply the chat template

In [ ]:
train_dataset[157]["text"]

In [ ]:
df = pd.read_csv("val.csv")

def parse_conversations(conv_str):
    try:
        return ast.literal_eval(conv_str)
    except:
        return []

# Transformer en liste de dictionnaires
dataset_list = [
    {"conversations": parse_conversations(conv)} for conv in df["conversations"]
]
hf_dataset_val = Dataset.from_list(dataset_list)
#hf_dataset=hf_dataset.select(range(100))



In [ ]:
#to hugging face formart(rule,content)
from unsloth.chat_templates import standardize_sharegpt
test_dataset = standardize_sharegpt( hf_dataset_val)
test_dataset = test_dataset.map(formatting_prompts_func, batched = True,)

<a name="Train"></a>
# **Train the model**
first we setup the experiment tracker wandb


In [ ]:
%%capture
import random
import wandb

# Start a new wandb run to track this script.
run = wandb.init(
    # Set the wandb entity where your project will be logged (generally your team name).
    entity="kimx94347-log",
    # Set the wandb project where this run will be logged.
    project="finetuning_llama2",
    # Track hyperparameters and run metadata.

)


In [ ]:
from trl import SFTTrainer,SFTConfig
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
        model=model,
        tokenizer=tokenizer,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        dataset_text_field="text",  # Adjust if your field name is different
        max_seq_length=max_seq_length,
        data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer),
        dataset_num_proc=2,
        #group_by_length=True,
        packing=False,  # Can make training 5x faster for short sequences
        args=SFTConfig(
            per_device_train_batch_size=8,
            per_device_eval_batch_size=8,
            gradient_accumulation_steps=4,
            dataloader_drop_last=True,
           # max_grad_norm=2.0,
            warmup_steps=100,
            num_train_epochs=3,
            learning_rate=5e-5,
            eval_strategy="steps",
            eval_steps=20,
            fp16=not is_bfloat16_supported(),
            bf16=is_bfloat16_supported(),
            logging_steps=1,
            optim="Adamw_8bit",
            weight_decay=0.01,
            lr_scheduler_type="cosine",
            seed=3407,
            output_dir="outputs_mistral",  # Path to save model checkpoints
            save_strategy="epoch",  # Save model after each epoch
            save_total_limit=7,  # Keep only the last 3 checkpoints
            report_to="wandb",  # Use this for WandB etc.
        ),
  )

# **extra steps to train on responses only(calculate loss only on the responses part)**

In [ ]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n",
)

test if the function is working correctly

In [ ]:
tokenizer.decode(trainer.train_dataset[48]["input_ids"])

In [ ]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[48]["labels"]])

We can see the Instruction prompts are successfully masked!

In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

# **Start the trainer**

In [ ]:
trainer_stats = trainer.train()

In [ ]:
from huggingface_hub import login
from transformers import AutoModelForCausalLM, AutoTokenizer


model_name="kimxxxx/llama_r32_64_b8_gas4_lr5e-5_4500tk_3epoch"
model_location="outputs_mistral/checkpoint-1164"
model = AutoModelForCausalLM.from_pretrained(model_location)
tokenizer = AutoTokenizer.from_pretrained(model_location)

model.push_to_hub(model_name)
tokenizer.push_to_hub(model_name)